<a href="https://colab.research.google.com/github/st4rmy/ML/blob/main/Zadanie4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install langchain sentence-transformers faiss-cpu pypdf transformers torch langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Successfully uninstalled langchain-core-0.3.74
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
g

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from langchain.chains import RetrievalQA

In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdf_paths = [
    "/content/drive/MyDrive/lalka.pdf",
    "/content/drive/MyDrive/przedwiosnie.pdf",
    "/content/drive/MyDrive/liryka.pdf"
]
all_pages = []
for path in pdf_paths:
    loader = PyPDFLoader(path)
    pages = loader.load()
    all_pages.extend(pages)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)
docs = text_splitter.split_documents(all_pages)

for idx,doc in enumerate(docs):
  print(f"[{idx}].{doc}\n\n")

[0].page_content='Autor opracowania: prof. dr hab. Marek Bernacki, ATH w Bielsku-Białej 
Opracowania ukazały się drukiem w 2002 roku w warszawskim Wydawnictwie 
ADAMANTAN w serii "Przeczytaj Przed Maturą". 
 
Bolesław Prus, Lalka (1890) 
 
Biogram autora' metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2020-04-02T20:23:43+02:00', 'title': 'Prus.Lalka', 'author': 'Marek Bernacki', 'moddate': '2020-04-02T20:23:43+02:00', 'source': '/content/drive/MyDrive/lalka.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}


[1].page_content='ADAMANTAN w serii "Przeczytaj Przed Maturą". 
 
Bolesław Prus, Lalka (1890) 
 
Biogram autora 
Bolesław Prus, właśc. Aleksander Głowacki (1847 -1912) – powieściopisarz, nowelista, 
publicysta. Wybitny przedstawiciel polskiego pozytywizmu.' metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2020-04-02T20:23:43+02:00', 'title': 'Prus.Lalka', 'author': 'Marek Bernac

Na samym początku ładuję pliki, na podstawie których model będzie miał za zadanie wygenerować odpowiedzi. Aby było to możliwe, ważne jest, żeby podzielono tekst na mniejsze fragmenty.

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

model_name = "sentence-transformers/distiluse-base-multilingual-cased-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

db = FAISS.from_documents(docs, embeddings)
db.save_local("faiss_index_hf")

/tmp/ipython-input-2050888540.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Tworzę bazę wektorową z dokumentów, które zostały podzielone na fragmenty przekształcone w wektory reprezentujące znaczenie poszczególnych części

In [12]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from langchain.chains import RetrievalQA

model_name = "dkleczek/Polish_BART_base_OPI"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024
)

llm = HuggingFacePipeline(pipeline=pipe)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)
while True:
    query = input("Użytkownik: ")

    if query.lower() == "exit":
        break

    result = qa_chain.invoke(query)
    print(f"AI Assistant: {result['result']}\n")

Device set to use cpu


Użytkownik: Czym jest "Melodia mgieł nocnych"?
AI Assistant: Use the following pieces of context to answer the question at the end. If you don't know the ansver, just say that you can't make up an answers.  Uwaga: o roli, jaką pełni w powieści pamiętnik „starego subiekta” – zob. Forma. Narracja). • opis przyrody – pejzażu tatrzańskiego – opisują go upersonifikowane mgły, które prezentują swoje wrażenia;którzy leżą tam na śm ietniku i może dlatego są nędzni i będą jeszcze nędzniejsi, że ja chcę wydawać po trzydzieści tysięcy rubli na zabawę w motyla. Głupi handlarzu, podły człowieku!”. W Wokulskim idealista bezustannie walczy z racjonalistą,  pozytywista z romantykiem,czyli przełom XVIII i XIX wieku,. () „Melodia mgieł nocnych” Kazimierz Przerwa – Tetmajer  (• Opis przyrody – krajobraz tatrzański, ) (((O roli w powieści Pamiętnik „Starego subiegta” - zob. forma. Narra).  ·Opis przyrody. · Postaci pierwszoplanowe• zjawiska tranzytywne – spadająca gwiazda, chwila przejścia nocy w dzień;  

W wyborze modelu kierowałam się głównie tym, jaki efekt chciałabym osiągnąć oraz faktem, że moje dokumenty są w języku polskim, co zmniejszyło liczbę modeli, które mogłabym wykorzystać.